In [1]:
# Setup packages 
from collections import Counter
from dash_bootstrap_components._components.Row import Row
import pandas as pd
from pandas.core.common import flatten
import numpy as np
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
import networkx.algorithms
import networkx.utils # (algorithms to check out: "approximation", "eccentricity", "diameter", "radius", "periphery", "center", "barycenter", "Community" "degree_centrality", "constraint", "local_constraint", "effective_size") 

# Dash packages needed for building dash app 

import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_cytoscape as cyto
import dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

# setup layout and paths
path = "/home/teijehidde/Documents/Git Blog and Coding/"
data_file = "data_dump/data_new6.json" 
external_stylesheets = path + 'Comparing Wikipedia Knowledge Networks (Network Analysis Page links)/Code/stylesheet.css' # downloaded from: https://codepen.io/chriddyp/pen/bWLwgP.css. Should appear in credits! 
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}
colors = {
    'background': 'white', # use color coding later. 
    'text': 'gray'
}
list_colours = ['red', 'blue', 'purple','orange','green','olive', 'maroon', 'brown','lime','teal' ]

In [2]:
# loading data. 
network_data_df = pd.read_json((path + data_file), orient='split')
all_networks = network_data_df.loc[network_data_df['ego'] == True].loc[network_data_df['lang'] == 'en']['title'].values.tolist()

In [3]:
# setting up classes WikiNode and WikiNetwork. 
class WikiNode:
    def __init__(self, node_title, lang, network_data):

        node_data = network_data.loc[network_data['title'] == node_title].loc[network_data['lang'] == lang]
        
        self.node_title = node_data[['title']].iloc[0,0] # iloc[0,0] needed because there can be two instance of same wikipage in the dataframe: one as centralnode (with langlinks) and one as a normal node of other network (without langlinks).  
        self.node_ID = node_data[['uniqueid']].iloc[0,0]
        self.node_links = node_data[['links']].iloc[0,0]
        self.node_lang = node_data[['lang']].iloc[0,0]

In [4]:
class WikiNetwork(WikiNode):
   
    def __init__(self, node_title, lang, threshold = 0):
        
        WikiNode.__init__(self, node_title, lang, network_data = network_data_df)
        self.threshold = threshold
        self.network_nodes = {}
        self.network_links = [node_title]
        self.network_edges = [] 
        self.network_status = []
        
        # Go through node_links of the central node (node_title) to build network.
        for link in self.node_links + [self.node_title]:
            try: 
                Node2 = WikiNode(link, lang, network_data = network_data_df) # NB: the links are not always in the same language as the network. It throws an error as result. - for now it just skips. 
                purged_links = [x for x in Node2.node_links if x in self.node_links]
                purged_edges = []
                for purged_link in purged_links:
                    purged_edges.append((link,purged_link))  
                self.network_nodes[Node2.node_ID] = Node2
                self.network_links = self.network_links + purged_links
                self.network_edges = self.network_edges + purged_edges
            except: 
                pass
        self.links_count = Counter(self.network_links)

    def getNodes(self, type="cytoscape"):
        selected_nodes = [k for k,v in self.links_count.items() if float(v) >= self.threshold]
        
        if type == 'networkx':
            return [(i, {"name": i}) for i in selected_nodes]
        if type == 'cytoscape':
            return [{'data': {'id': i, "label": i}} for i in selected_nodes]

    def getEdges(self,type="cytoscape"):  
        selected_nodes = [k for k,v in self.links_count.items() if float(v) >= self.threshold]
        edges_network = [(a,b) for a,b in self.network_edges if a in selected_nodes and b in selected_nodes]
        
        if type == 'networkx':
            return edges_network
        if type == 'cytoscape':
            return [{'data': {'source': a, "target": b}} for a,b in edges_network]

    def getStatsNodes(self):
        G = nx.Graph()
        G.add_edges_from(self.getEdges(type = 'networkx'))
        centrality_nodes = networkx.algorithms.centrality.eigenvector_centrality(G)
        # eccentricity_nodes = networkx.algorithms.distance_measures.eccentricity(G)
        df = pd.DataFrame({'network_centrality':pd.Series(centrality_nodes)}) #  'eccentricity':pd.Series(eccentricity_nodes) 

        val_max = max(df['network_centrality'])
        val_min = min(df['network_centrality'])
        df[['centrality_normalized']] = df[['network_centrality']].apply(lambda x: (x - val_min) / (val_max - val_min), result_type = 'expand')
        df[['centrality_rounded']] = df[['network_centrality']].apply(lambda x: round(x, 2))

        return df

    def getStatsCommunities(self):
        G = nx.Graph()
        G.add_edges_from(self.getEdges(type = 'networkx'))
        communities = greedy_modularity_communities(G)
        dict_communities = {key:value for value in range(len(communities)) for key in communities[value] }

        community_centrality_nodes = {}
        for community in communities:
            selected_edges = [(a,b) for a,b in G.edges if a in community if b in community]
            G_community = nx.Graph()
            G_community.add_edges_from(selected_edges)
            community_centrality_nodes.update(networkx.algorithms.centrality.degree_centrality(G))
            
        return pd.DataFrame({'community':pd.Series(dict_communities), 'community_centrality': pd.Series(community_centrality_nodes)}) 


In [5]:
network_data_df.loc[network_data_df['title'] == 'لقاح']

,title,lang,pageid,uniqueid,lastrevid,links,langlinks,ego
2503,لقاح,ar,597715,ar597715,54578135,"[Ad26.COV2.S, Ad5-nCOV, BBV152, BRW (magazine)...","[{'lang': 'af', '*': 'Entstof'}, {'lang': 'ak'...",1.0


In [6]:
wiki_page = WikiNetwork('لقاح', 'ar')

In [27]:
nodes = wiki_page.getNodes(type='cytoscape')
edges = wiki_page.getEdges(type='cytoscape')
stats_nodes = wiki_page.getStatsNodes()
stats_communities = wiki_page.getStatsCommunities()
pd_nodes = pd.concat([stats_nodes, stats_communities], axis = 1).reset_index()
pd_nodes.rename('index': 'title')

stats_nodes = {'node_title': 'لقاح', 'lang': 'ar', 'nodes_network': nodes, 'edges_network': edges, 'nodes_stats': pd_nodes.to_dict('records')} 

In [28]:
pd_nodes

,index,network_centrality,centrality_normalized,centrality_rounded,community,community_centrality
0,Ad5-nCOV,0.029839,0.220337,0.03,1,0.247761
1,Ad26.COV2.S,0.028962,0.213615,0.03,1,0.238806
2,BBV152,0.029839,0.220337,0.03,1,0.247761
3,إبيفاك كورونا,0.030260,0.223556,0.03,1,0.250746
4,تجربة سريرية,0.049902,0.374040,0.05,1,0.346269
...,...,...,...,...,...,...
331,Betz Halloran,0.001080,0.000000,0.00,0,0.002985
332,Clin. Infect. Dis.,0.001080,0.000000,0.00,0,0.002985
333,أنثى البقرة,0.001080,0.000000,0.00,0,0.002985
334,القاموس الجهنمي، الطبعة السادسة,0.001080,0.000000,0.00,0,0.002985


In [12]:
pd.DataFrame.from_dict(stats_nodes['nodes_stats']) 

,page_ID,title,network_centrality,centrality_normalized,centrality_rounded,community,community_centrality
0,ar8338317,Ad26.COV2.S,0.028962,0.213615,0.03,1,0.238806
1,ar8030088,Ad5-nCOV,0.029839,0.220337,0.03,1,0.247761
2,ar8030084,BBV152,0.029839,0.220337,0.03,1,0.247761
3,ar2588132,Open access,0.001333,0.001941,0.00,1,0.005970
4,ar6874832,أبحاث فيروس الهربس البسيط,0.028765,0.212103,0.03,1,0.226866
...,...,...,...,...,...,...,...
331,NaN,NaN,0.001080,0.000000,0.00,0,0.002985
332,NaN,NaN,0.001080,0.000000,0.00,0,0.002985
333,NaN,NaN,0.001080,0.000000,0.00,0,0.002985
334,NaN,NaN,0.001080,0.000000,0.00,0,0.002985


In [10]:
int(stats_nodes.loc['وبائيات']['community'])

AttributeError: 'dict' object has no attribute 'loc'

In [62]:
nodes = wiki_page.getNodes(type='cytoscape')
edges = wiki_page.getEdges(type='cytoscape')
stats_nodes = wiki_page.getStatsNodes()
stats_communities = wiki_page.getStatsCommunities()

pd_nodes = pd.DataFrame([{'page_ID': v.node_ID, 'title': v.node_title} for v in wiki_page.network_nodes.values()]).set_index('title', drop = False)
pd_nodes = pd.concat([pd_nodes, stats_nodes, stats_communities], axis = 1)

data = {'nodes_network': nodes, 'edges_network': edges, 'nodes_stats': pd_nodes.to_dict('records')} 

In [65]:
stats_nodes = pd.DataFrame.from_dict(data['nodes_stats']) 
stats_nodes = stats_nodes.set_index('title', drop = False)

In [66]:
stats_nodes

,page_ID,title,network_centrality,centrality_normalized,centrality_rounded,community,community_centrality
title,,,,,,,
2009 swine flu pandemic vaccine,en23224587,2009 swine flu pandemic vaccine,0.316393,0.315271,0.32,0,0.316393
2017 Democratic Republic of the Congo Ebola virus outbreak,en54039081,2017 Democratic Republic of the Congo Ebola vi...,0.016393,0.014778,0.02,0,0.016393
ABC News (Australia),en4797328,ABC News (Australia),0.004918,0.003284,0.00,0,0.004918
ACE inhibitor,en2767,ACE inhibitor,0.283607,0.282430,0.28,1,0.283607
AIDS,en36056314,AIDS,0.029508,0.027915,0.03,0,0.029508
...,...,...,...,...,...,...,...
Zika virus vaccine,en53907564,Zika virus vaccine,0.157377,0.155993,0.16,0,0.157377
Zoster vaccine,en8125462,Zoster vaccine,0.162295,0.160920,0.16,0,0.162295
Vaccine,en32653,Vaccine,1.000000,1.000000,1.00,0,1.000000


In [68]:
int(stats_nodes.loc['ACE inhibitor']['community'])

1